In [1]:
from qrisp import QuantumFloat,h
from qrisp.interface.provider_backends.planqk_backend import PlanQKBackend
token = "plqk_G7YWFsDxNINX6LIPp2aZIWqbzizl3lm7c7cIydUaeo"
org_id = "bddb80ee-148a-4e4b-98b1-e6fda7c26d4c"

For these examples we instanciate a Quantumfloat as an int =2 (in binary  "0100"), apply an h-gate on the first qubit, s.t. the state is {0: 0.5, 2: 0.5} (in binary {"0000": 0.5, "0100": 0.5})  and multiply it with itself. So we would expect a dictionary to return {0: 0.5, 4: 0.5} (in binary {"0000": 0.5, "0010": 0.5}). 
See the notebook with results for individual simulators below


A general problem is that unmeasured ancillas are returned by default aswell
- messes with the encoding in qrisp
- does not allow for circuits to be executed, in which not all qubits are measured. This a functionality we need, as this multiplication procedure includes such qubits (as do many other algorithms)

In [2]:

# WORKS
# aws.sim.sv1, aws.sim.dm1  WORKS with [::-1] on the returned measurement dictionary --> reversed results      
# we get the reversed result which encodes a 4 in the first 4 qubits, plus 4 ancilla qubits that irrelevant {'0010010': 470, '0000000': 530}  
# this translates to {0: 0.53, 4: 0.47}, if we cut out the non-measured registers.
# 
example_backend = PlanQKBackend(api_token = token, device_instance = "aws.sim.sv1", organization_id=org_id) 

qf = QuantumFloat(2)
qf[:] = 2
h(qf[1])
res = qf*qf
print(qf.qs)
result = res.get_measurement(backend = example_backend)
print(result)

QuantumCircuit:
---------------
                     ┌──────────┐
     qf.0: ──────────┤0         ├
           ┌───┐┌───┐│          │
     qf.1: ┤ X ├┤ H ├┤1         ├
           └───┘└───┘│          │
mul_res.0: ──────────┤2         ├
                     │          │
mul_res.1: ──────────┤3 __mul__ ├
                     │          │
mul_res.2: ──────────┤4         ├
                     │          │
mul_res.3: ──────────┤5         ├
                     │          │
sbp_anc.0: ──────────┤6         ├
                     └──────────┘
Live QuantumVariables:
----------------------
QuantumFloat qf
QuantumFloat mul_res


c:\Users\nik40643\Desktop\newVersion\Qrisp\.venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:609: UserWarning: Device does not support global phase; global phase of 4.2215151282612835 will not be included in Braket circuit
  warnings.warn(
Request post https://platform.planqk.de/quantum/jobs failed (Trace a076a7291303191455621c59ee4f9d13): 400 Client Error:  for url: https://platform.planqk.de/quantum/jobs


PlanqkClientError: Invalid job input data or parameter provided for backend "aws.sim.sv1": Device requires all qubits in the program to be measured. This may be caused by declaring non-contiguous qubits or measuring partial qubits (Service: Braket, Status Code: 400, Request ID: db8a1dac-4da3-4540-9112-527a911e2e08) (SDK Attempt Count: 1) (HTTP error: 400)

In [ ]:

# RUNS, but faulty results
# azure.ionq.simulator gives faulty result, even after reverse
# aws.rigetti.ankaa too much crosstalk 
# qryd.sim.square gives faulty result, even after reverse -- {'0000000': 490, '000001001': 510}
# qryd.sim.square gives faulty result, even after reverse -- {'0000000': 484, '000001001': 516}
# aws.iqm.garnet too much crosstalk

from qrisp import QuantumVariable, rx,cx 
import numpy as np
example_backend = PlanQKBackend(api_token = token, device_instance = "qryd.sim.square", organization_id=org_id) 
qv = QuantumVariable(4)
#q2 = QuantumVariable(2)

h(qv(0))
rx(np.pi/2,qv(1) )
cx(qv(0), [qv(2),qv(3)])

result = res.get_measurement(backend = example_backend)
print(result)

In [ ]:

example_backend = PlanQKBackend(api_token = token, device_instance = "ibm.qpu.fez", organization_id=org_id) 

# DOESNT 
# all ibm qpus --> "Input [type of circuit object that is handed to the backend] should be ... (QISKIT)", is:  "QISKIT_QPY"
# maybe a planqk issue?

qf = QuantumFloat(2)

qf[:] = 2
h(qf[1])
res = qf*qf
result = res.get_measurement(backend = example_backend)
print(result)